# Import Python Packages

In [1]:
# Import Python packages 
import datetime
import pandas as pd
import os
import glob
import psycopg2

# Connection Set up 

## DB Params

In [10]:
DB_ENDPOINT = "csvtoazurepostgres.postgres.database.azure.com"
DEFAULT_DB = 'postgres'
DB = 'csv_etl'
DB_USER = 'tempUser_accident_data_ETL'
DB_PASSWORD = 'A7.Kr2phuVjagj3'
DB_PORT = '5432'

## Create a connection to Default Database

In [4]:
try: 
    conn = psycopg2.connect(
                        host=DB_ENDPOINT,
                        port=DB_PORT,
                        dbname=DEFAULT_DB,
                        user=DB_USER,
                        password=DB_PASSWORD)
    print("Connection established")
    conn.set_session(autocommit=True)
    print("Connection is in Auto Commit")
except psycopg2.Error as e: 
    print("Error: Could not make connection to the Default Postgres database")
    print(e)

Connection established
Connection is in Auto Commit


## Create Cursor

In [5]:
try: 
    cursor = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

## Create New DB and reset connection 

In [14]:
query = f"DROP DATABASE IF EXISTS {DB}"
try: 
    cursor.execute(query)
    query = f"CREATE DATABASE {DB}"
    cursor.execute(query)
except psycopg2.Error as e: 
    print("Error: Issue creating DataBase")
    print (e)

### Reset connection to the Desired DB

In [16]:
cursor.close()
conn.close()
try: 
    conn = psycopg2.connect(
                        host=DB_ENDPOINT,
                        port=DB_PORT,
                        dbname=DB,
                        user=DB_USER,
                        password=DB_PASSWORD)
    print("Connection established")
    conn.set_session(autocommit=True)
    print("Connection is in Auto Commit")
except psycopg2.Error as e: 
    print(f"Error: Could not make connection to the {DB} database")
    print(e)
    
try: 
    cursor = conn.cursor()
except psycopg2.Error as e: 
    print("Error: Could not get cursor to the Database")
    print(e)

Connection established
Connection is in Auto Commit


# Clean DATA , DROP unconstant Data, DROP Unused DATA

In [70]:
# Show numeric output in decimal format e.g., 2.15
pd.options.display.float_format = '{:,.2f}'.format
data_frame_riders = pd.read_csv('./data/riders.csv',header=None,names=['rider_id','first','last','address','birthday','account_start_date','account_end_date','is_member'])
data_frame_payments = pd.read_csv('./data/payments.csv')
data_frame_stations = pd.read_csv('./data/stations.csv')
data_frame_trips = pd.read_csv('./data/trips.csv')

## Explore the Data & Check for Consistency

### Riders CSV 

In [71]:
print(data_frame_riders.shape)
print(data_frame_riders.info())
print(data_frame_riders.sample())

(75000, 8)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   rider_id            75000 non-null  int64 
 1   first               75000 non-null  object
 2   last                75000 non-null  object
 3   address             75000 non-null  object
 4   birthday            75000 non-null  object
 5   account_start_date  75000 non-null  object
 6   account_end_date    14954 non-null  object
 7   is_member           75000 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 4.1+ MB
None
       rider_id        first   last                      address    birthday  \
43551     44551  Christopher  Jones  645 Ross Junctions Apt. 653  2001-06-19   

      account_start_date account_end_date  is_member  
43551         2019-10-24              NaN       True  


In [72]:
print(f'Missing values for data set?: {data_frame_riders.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_riders.isna().values.sum()}')
print(data_frame_riders.isna)
data_frame_riders['account_end_date']  = data_frame_riders['account_end_date'] .fillna(0)
print(f'Missing values for data set?: {data_frame_riders.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_riders.isna().values.sum()}')
print(data_frame_riders.isna)
#data_frame_riders.dropna(inplace=True)
# Remove duplicates
print(f'duplicates values for data set?: {data_frame_riders.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame_riders.duplicated().values.sum()}')
#print(data_frame_riders.duplicated)  

Missing values for data set?: True
Missing values for data set?: 60046
<bound method DataFrame.isna of        rider_id       first       last                        address  \
0          1000       Diana      Clark            1200 Alyssa Squares   
1          1001    Jennifer      Smith                397 Diana Ferry   
2          1002       Karen      Smith      644 Brittany Row Apt. 097   
3          1003       Bryan    Roberts         996 Dickerson Turnpike   
4          1004       Jesse  Middleton         7009 Nathan Expressway   
...         ...         ...        ...                            ...   
74995     75995  Alexandria      Smith    8045 Amanda Union Suite 942   
74996     75996      Nicole     Watson               5255 Andrea View   
74997     75997       James   Martinez  88210 Sheryl Islands Apt. 191   
74998     75998       Nancy      Watts      0175 Mary Hollow Apt. 187   
74999     75999        Ross      Green           8086 Fernandez Drive   

         birthday ac

#### Check  data after cleanning

In [73]:
print(data_frame_riders.shape)
print(data_frame_riders.sample())

(75000, 8)
       rider_id   first     last                         address    birthday  \
16772     17772  Victor  Stewart  7649 Kristen Mission Suite 456  1953-01-08   

      account_start_date account_end_date  is_member  
16772         2021-12-22                0       True  


#### Change Columns type to the desired format

In [74]:
print(data_frame_riders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   rider_id            75000 non-null  int64 
 1   first               75000 non-null  object
 2   last                75000 non-null  object
 3   address             75000 non-null  object
 4   birthday            75000 non-null  object
 5   account_start_date  75000 non-null  object
 6   account_end_date    75000 non-null  object
 7   is_member           75000 non-null  bool  
dtypes: bool(1), int64(1), object(6)
memory usage: 4.1+ MB
None


In [75]:
data_frame_riders['account_end_date']  = pd.to_datetime(data_frame_riders['account_end_date'])
data_frame_riders['birthday']  = pd.to_datetime(data_frame_riders['birthday'])
data_frame_riders['account_start_date']  = pd.to_datetime(data_frame_riders['account_start_date'])

#### See the new data types and if the data is ready

In [76]:
print(data_frame_riders.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75000 entries, 0 to 74999
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   rider_id            75000 non-null  int64         
 1   first               75000 non-null  object        
 2   last                75000 non-null  object        
 3   address             75000 non-null  object        
 4   birthday            75000 non-null  datetime64[ns]
 5   account_start_date  75000 non-null  datetime64[ns]
 6   account_end_date    75000 non-null  datetime64[ns]
 7   is_member           75000 non-null  bool          
dtypes: bool(1), datetime64[ns](3), int64(1), object(3)
memory usage: 4.1+ MB
None


### Payments CSV 

In [ ]:
print(data_frame_payments.shape)
print(data_frame_payments.columns)
print(data_frame_payments.sample())

In [ ]:
print(f'Missing values for data set?: {data_frame_payments.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_payments.isna().values.sum()}')
#data_frame_payments.dropna(inplace=True)
print(f'Missing values for data set?: {data_frame_payments.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_payments.isna().values.sum()}')
# Remove duplicates
print(f'duplicates values for data set?: {data_frame_payments.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame_payments.duplicated().values.sum()}')
print(data_frame_payments.duplicated)  

#### Check  data after cleanning

In [ ]:
print(data_frame_payments.shape)
print(data_frame_payments.columns)
print(data_frame_payments.sample())

#### Change Columns type to the desired format

In [ ]:
print(data_frame_payments.describe())
print(data_frame_payments.info())

### Stations CSV 

In [ ]:
print(data_frame_stations.shape)
print(data_frame_stations.columns)
print(data_frame_stations.sample())

In [ ]:
print(f'Missing values for data set?: {data_frame_stations.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_stations.isna().values.sum()}')
#data_frame_stations.dropna(inplace=True)
print(f'Missing values for data set?: {data_frame_stations.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_stations.isna().values.sum()}')
# Remove duplicates
print(f'duplicates values for data set?: {data_frame_stations.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame_stations.duplicated().values.sum()}')
print(data_frame_stations.duplicated)  

#### Check  data after cleanning

In [ ]:
print(data_frame_stations.shape)
print(data_frame_stations.columns)
print(data_frame_stations.sample())

#### Change Columns type to the desired format

In [ ]:
print(data_frame_stations.describe())
print(data_frame_stations.info())

### Trips CSV 

In [ ]:
print(data_frame_trips.shape)
print(data_frame_trips.columns)
print(data_frame_trips.sample())

In [ ]:
print(f'Missing values for data set?: {data_frame_trips.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_trips.isna().values.sum()}')
#data_frame_trips.dropna(inplace=True)
print(f'Missing values for data set?: {data_frame_trips.isna().values.any()}')
print(f'Missing values for data set?: {data_frame_trips.isna().values.sum()}')
# Remove duplicates
print(f'duplicates values for data set?: {data_frame_trips.duplicated().values.any()}')
print(f'duplicates values for data set?: {data_frame_trips.duplicated().values.sum()}')
print(data_frame_trips.duplicated)  

#### Check  data after cleanning

In [ ]:
print(data_frame_trips.shape)
print(data_frame_trips.columns)
print(data_frame_trips.sample())

#### Change Columns type to the desired format

In [ ]:
print(data_frame_trips.describe())
print(data_frame_trips.info())

# Populating Cleaned Data 

## Automation Functions

In [ ]:
# Helper functions
def table_recreate(cursor, tableName: str, tableFields: str ):
     """ Takes the Connection , Table name and create Fields, will drop the table if exists and create it again with the desired fields Ex : table_recreate(cursor, payments,
                    "(sessionId INT,
                    itemInSession INT,
                    artist TEXT,
                    song TEXT,
                    length FLOAT, 
                    PRIMARY KEY (sessionId, itemInSession))")  """
    try:
        
        cursor.execute("DROP TABLE IF EXISTS {0};".format(tableName))
        query = f"CREATE TABLE {tableName} "
        query = query + tableFields
        cursor.execute(query)
        print("Finished creating table {0}".format(tableName))
    except psycopg2.Error as e: 
    print(f"Error: Couldn't recreate the table: {tableName}, something went wrong")
    print(e)

## Create Tables & Insert data

### Riders Table 

#### Create

In [ ]:
TABLE = "riders"
FIELDS = """(rider_id INTEGER PRIMARY KEY,
                    first VARCHAR(50),
                    last VARCHAR(50), 
                    address VARCHAR(100), 
                    birthday DATE, 
                    account_start_date DATE, 
                    account_end_date DATE, 
                    is_member BOOLEAN);"""
table_recreate(cursor = cursor, tableName = TABLE, tableFields = FIELDS )

#### Populate

In [ ]:
for index, row in data_frame_riders.iterrows():   
    query = "INSERT INTO riders (rider_id, first, last, address, birthday, account_start_date, account_end_date, is_member)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["rider_id"], row["first"], row["last"], row["birthday"], row["account_start_date"], row["account_end_date"], row["is_member"]))
    except Exception as e:
        print(e)
Print("Finished Populating ")

### Payments Table 

#### Create

In [ ]:
TABLE = "payment"
FIELDS = """(payment_id INTEGER PRIMARY KEY, 
                    date DATE, 
                    amount MONEY, 
                    rider_id INTEGER, );"""
table_recreate(cursor = cursor, tableName = TABLE, tableFields = FIELDS )

#### Populate

In [ ]:
for index, row in data_frame_riders.iterrows():   
    query = "INSERT INTO payment (payment_id, date, amount, rider_id)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["payment_id"], row["date"], row["amount"], row["rider_id"]))
    except Exception as e:
        print(e)
Print("Finished Populating ")

### Stations Table 

#### Create

In [ ]:
TABLE = "station"
FIELDS = """(station_id VARCHAR(50) PRIMARY KEY, 
                    name VARCHAR(75), 
                    latitude FLOAT, 
                    longitude FLOAT, );"""
table_recreate(cursor = cursor, tableName = TABLE, tableFields = FIELDS )

#### Populate

In [ ]:
for index, row in data_frame_riders.iterrows():   
    query = "INSERT INTO station (station_id, name, latitude, longitude)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["station_id"], row["name"], row["latitude"], row["longitude"]))
    except Exception as e:
        print(e)
Print("Finished Populating ")

### Trips Table 

#### Create

In [ ]:
TABLE = "trip"
FIELDS = """(trip_id VARCHAR(50) PRIMARY KEY, 
                    rideable_type VARCHAR(75), 
                    start_at TIMESTAMP, 
                    ended_at TIMESTAMP,
                    start_station_id VARCHAR(50),
                    end_station_id VARCHAR(50),
                    rider_id INTEGER, );"""
table_recreate(cursor = cursor, tableName = TABLE, tableFields = FIELDS )

#### Populate

In [ ]:
for index, row in data_frame_riders.iterrows():   
    query = "INSERT INTO trip (trip_id, rideable_type, start_at, ended_at, start_station_id, end_station_id, rider_id)"
    query = query + "VALUES (%s, %s, %s, %s, %s)"
    try:
        session.execute(query, (row["trip_id"], row["rideable_type"], row["start_at"], row["ended_at"], row["start_station_id"], row["end_station_id"], row["rider_id"]))
    except Exception as e:
        print(e)
Print("Finished Populating ")